In [1]:
import Statistics

In [36]:
# Generate data
features = rand(100,4)
target = randn(100)
target .= target .>= 0 
data = [features  target]

100×5 Matrix{Float64}:
 0.275182   0.444251   0.632674    0.828238  1.0
 0.391216   0.345759   0.646216    0.752417  1.0
 0.357564   0.603281   0.760644    0.611507  0.0
 0.4922     0.584713   0.727695    0.607048  0.0
 0.129269   0.908936   0.302889    0.140603  0.0
 0.0510008  0.0592317  0.00744346  0.822262  1.0
 0.249976   0.0417189  0.444894    0.875329  0.0
 0.523887   0.32305    0.220975    0.128879  0.0
 0.853443   0.416895   0.142651    0.150012  1.0
 0.98796    0.971922   0.22905     0.103246  1.0
 ⋮                                           
 0.155655   0.499991   0.00258236  0.66141   1.0
 0.554915   0.59908    0.118149    0.746277  1.0
 0.393676   0.870807   0.935769    0.816109  0.0
 0.433669   0.432481   0.931418    0.81516   1.0
 0.653601   0.527884   0.0385365   0.609924  0.0
 0.421326   0.33974    0.989121    0.316383  1.0
 0.470162   0.682437   0.289545    0.84587   0.0
 0.908293   0.945891   0.893266    0.348585  1.0
 0.658054   0.885958   0.886233    0.446441  1.0

In [3]:
size(features)

(100, 4)

In [4]:
function seprate_classes(x,y) # Separate the dataset into a subset of data for each class
    seprated_classes = Dict()
    for i in 1:size(x,1)
        feature = x[i,:]
        class_name = y[i]
        if class_name ∉ keys(seprated_classes) 
            seprated_classes[class_name] = []
        end
        append!(seprated_classes[class_name],feature)
    end
    return seprated_classes
end

seprate_classes (generic function with 1 method)

In [5]:
classes = seprate_classes(features,target)

Dict{Any, Any} with 2 entries:
  0.0 => Any[0.428984, 0.945018, 0.98762, 0.377946, 0.597905, 0.699222, 0.85448…
  1.0 => Any[0.747751, 0.0694416, 0.432497, 0.611331, 0.0935131, 0.00723967, 0.…

In [6]:
function std_mean(x) # Calculates standard deviation and mean of features.
    stat = Dict("std"=>[],"mean"=>[])
    for feature in eachcol(x)
        append!(stat["std"],Statistics.std(feature))
        append!(stat["mean"],Statistics.mean(feature))
    end
    stdmean = []
    for key in keys(stat)
        for val in stat[key]
            append!(stdmean,key=>val)
        end
    end
    return stdmean
end

std_mean (generic function with 1 method)

In [7]:
stdmean = std_mean(features)

16-element Vector{Any}:
  "mean"
 0.4133634074405287
  "mean"
 0.49731092372839797
  "mean"
 0.5271376204443912
  "mean"
 0.5230942395581106
  "std"
 0.2908688280985218
  "std"
 0.3036763904607861
  "std"
 0.2725333464432797
  "std"
 0.28445110977257027

In [8]:
stdmean |> typeof

Vector{Any} (alias for Array{Any, 1})

In [9]:
length(stdmean)

16

In [10]:
function distribution(x,std,mean) # Gaussian Distribution Function
    exponent = exp.(-((x .- mean).^2 ./ (2 .* std .^ 2)))
    return exponent ./ (sqrt(2*π)*std)
end

distribution (generic function with 1 method)

In [11]:
function fit(x,y)
    classes = seprate_classes(x,y)
    class_summary = Dict()
    for item in classes
        class_name , feature_val = item.first , item.second
        class_summary[class_name] = Dict("prior_proba"=>length(feature_val)/size(x,1),
                                        "summary"=>[i for i in std_mean(feature_val)])
    end
    return class_summary        
end

fit (generic function with 1 method)

In [12]:
class_summary = fit(features,target)

Dict{Any, Any} with 2 entries:
  0.0 => Dict{String, Any}("summary"=>Any["mean", 0.515391, "std", 0.282968], "…
  1.0 => Dict{String, Any}("summary"=>Any["mean", 0.466998, "std", 0.296369], "…

In [13]:
function predict(x,class_summary)
    MAPs = []
    for row in eachrow(x)
        joint_proba = Dict()
        for item in class_summary
            class_name , features = item.first,item.second
            total_features = length(features["summary"])
            likelihood =1

            for idx in 1:total_features
                feature = row[idx]
                mean_ = features["summary"][idx]["mean"]
                std_ = features["summary"][idx]["std"]
                normal_proba = distribution(feature,std_,mean_)
                likelihood *= normal_proba
            end
            prior_proba = features["prior_proba"]
            joint_proba[class_name] = prior_proba * likelihood
        end
        MAP = max((x,y)->joint_proba[x]>joint_proba[y] ? x : y,keys(joint_proba)) # ?
        append!(MAPs,MAP)
    end
    return MAPs
end

predict (generic function with 1 method)

In [14]:
maps = predict(features,class_summary)

MethodError: MethodError: no method matching getindex(::String, ::String)
Closest candidates are:
  getindex(::AbstractString, !Matched::Colon) at strings/basic.jl:189
  getindex(::String, !Matched::Int64) at strings/string.jl:226
  getindex(::AbstractString, !Matched::Integer) at strings/basic.jl:184
  ...

In [39]:
maps = []
for row in eachrow(features)
    for item in class_summary
        class_name , features_val = item.first,item.second
        # println(class_name)
        println(features_val)
        total_features = length(features_val["summary"])
        likelihood =1
        
        println("doroste")
        for idx in 1:total_features
            @show feature = row[idx]
            mean_ = features_val["summary"][idx]["mean"]
            std_ = features_val["summary"][idx]["std"]
            normal_proba = distribution(feature,std_,mean_)
            likelihood *= normal_proba
        end
        prior_proba = features_val["prior_proba"]
        joint_proba[class_name] = prior_proba * likelihood

        println("==============")
    end
end
    

Dict{String, Any}("summary" => Any["mean", 0.5153905088085918, "std", 0.28296832561538665], "prior_proba" => 1.92)
doroste
feature = row[idx] = 0.2751819096876341


MethodError: MethodError: no method matching getindex(::String, ::String)
Closest candidates are:
  getindex(::AbstractString, !Matched::Colon) at strings/basic.jl:189
  getindex(::String, !Matched::Int64) at strings/string.jl:226
  getindex(::AbstractString, !Matched::Integer) at strings/basic.jl:184
  ...

In [45]:
class_summary[0]["summary"]

4-element Vector{Any}:
  "mean"
 0.5153905088085918
  "std"
 0.28296832561538665

In [17]:
[item.first=>item.second[i] for (i,item) in enumerate(a)]

2-element Vector{Pair{String, Int64}}:
 "wwww" => 6
 "qqqq" => 2

In [18]:
for key in keys(a)
    for (i,item) in enumerate(a[key])
        println(key=>item)
    end
end

"wwww" => 6
"wwww" => 7
"wwww" => 8
"wwww" => 9
"qqqq" => 1
"qqqq" => 2
"qqqq" => 3
"qqqq" => 4


In [19]:
[i=>j for i in 1:5,j in 1:5]

5×5 Matrix{Pair{Int64, Int64}}:
 1=>1  1=>2  1=>3  1=>4  1=>5
 2=>1  2=>2  2=>3  2=>4  2=>5
 3=>1  3=>2  3=>3  3=>4  3=>5
 4=>1  4=>2  4=>3  4=>4  4=>5
 5=>1  5=>2  5=>3  5=>4  5=>5